# Basics II

**Check the following line. Why there is no GPU?**

In [13]:
import tensorflow as tf
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [14]:
# Get the current device
current_device = qibo.get_device()
print(f"Current device: {current_device}")

Current device: /GPU:0


In [2]:
import qibo

# Get the current backend
current_backend = qibo.get_backend()
print(f"Current backend: {current_backend}")

# Get the current device
current_device = qibo.get_device()
print(f"Current device: {current_device}")

[Qibo 0.2.1|INFO|2023-10-24 22:30:25]: Using tensorflow backend on /device:CPU:0


Current backend: tensorflow
Current device: /device:CPU:0


In [4]:
from qibo.models import QFT

qibo.set_backend("tensorflow")
qibo.set_device("/CPU:0")

c = QFT(20)
final_state = c() # circuit will now be executed on CPU
print()
print(final_state)

[Qibo 0.2.1|INFO|2023-10-24 22:30:47]: Using tensorflow backend on /CPU:0
[Qibo 0.2.1|INFO|2023-10-24 22:30:47]: Using tensorflow backend on /CPU:0



(0.00098+0j)|00000000000000000000> + (0.00098+0j)|00000000000000000001> + (0.00098+0j)|00000000000000000010> + (0.00098+0j)|00000000000000000011> + (0.00098+0j)|00000000000000000100> + (0.00098+0j)|00000000000000000101> + (0.00098+0j)|00000000000000000110> + (0.00098+0j)|00000000000000000111> + (0.00098+0j)|00000000000000001000> + (0.00098+0j)|00000000000000001001> + (0.00098+0j)|00000000000000001010> + (0.00098+0j)|00000000000000001011> + (0.00098+0j)|00000000000000001100> + (0.00098+0j)|00000000000000001101> + (0.00098+0j)|00000000000000001110> + (0.00098+0j)|00000000000000001111> + (0.00098+0j)|00000000000000010000> + (0.00098+0j)|00000000000000010001> + (0.00098+0j)|00000000000000010010> + (0.00098+0j)|00000000000000010011> + ...


In [10]:
from qibo import Circuit, gates
qibo.set_device("/CPU:0")

# Construct the circuit
c = QFT(25)
for i in range(25):
    c.add(gates.H(i))

final_state = c() # circuit will now be executed on CPU
print()
print(final_state)

[Qibo 0.2.1|INFO|2023-10-24 22:50:41]: Using tensorflow backend on /CPU:0



(1+0j)|0000000000000000000000000>


In [16]:
import numpy as np
import qibo
from qibo import Circuit, gates

qibo.set_device("/GPU:5")

# Construct the circuit
c = QFT(25)
for i in range(25):
    c.add(gates.H(i))

final_state = c() # circuit will now be executed on GPU
print()
print(final_state)

[Qibo 0.2.1|INFO|2023-10-25 00:54:56]: Using tensorflow backend on /GPU:5



(1+0j)|0000000000000000000000000>


## Restart Kernet for setting the number of CPU threads

Un "thread" (o hilo en español) es una secuencia de instrucciones que pueden ser ejecutadas en paralelo con otras secuencias en un programa. En un sistema operativo, los threads son una forma de permitir que múltiples operaciones ocurran simultáneamente dentro de un mismo proceso.

**From here, everything has turned very incomprehensible!**

In [1]:
import tensorflow as tf
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)
# Explore more on this!!

import qibo
from qibo.models import Circuit


qibo.set_device("/CPU:0")

# accelerators = {"/GPU:0": 3, "/GPU:1": 1}

c = Circuit(2)#, accelerators)
final_state = c() # circuit will now be executed on CPU
print(final_state)

[Qibo 0.2.1|INFO|2023-10-25 17:23:02]: Using tensorflow backend on /device:CPU:0
[Qibo 0.2.1|INFO|2023-10-25 17:23:02]: Using tensorflow backend on /CPU:0


(1+0j)|00>


In [4]:
# set the number of threads to 1
qibo.set_threads(1)
# retrieve the current number of threads
current_threads = qibo.get_threads()

[Qibo 0.2.1|WARNING|2023-10-24 20:37:35]: `set_threads` is not supported by the tensorflow backend. Please use tensorflow's thread setters: `tf.config.threading.set_inter_op_parallelism_threads` or `tf.config.threading.set_intra_op_parallelism_threads` to switch the number of threads.


In [17]:
# import qibo
# import numpy as np

# qibo.set_backend("numpy")

# from qibo import Circuit, gates

# qibo.set_device("/CPU:0")

accelerators = {"/GPU:0": 3, "/GPU:1": 1}

c = Circuit(5, accelerators)


c.add(gates.X(0))
c.add(gates.X(1))
c.add(gates.CU1(0, 1, 0.1234))


init_state = np.ones(4) / 2.0
c(init_state)

[Qibo 0.2.1|ERROR|2023-10-25 01:23:32]: tensorflow does not support distributed execution.


NotImplementedError: tensorflow does not support distributed execution.

**Up to here, everything has been very confusing!**

## Callbacks for entanglement entropy

An example use case of this is the calculation of entanglement entropy as the state propagates through a circuit. This can be implemented easily using `qibo.callbacks.EntanglementEntropy` and the `qibo.gates.CallbackGate` gate.

In [31]:
from qibo import models, gates, callbacks

# create entropy callback where qubit 0 is the first subsystem
entropy = callbacks.EntanglementEntropy([0])

# initialize circuit with 2 qubits and add gates
c = models.Circuit(2) # state is |00> (entropy = 0)
c.add(gates.CallbackGate(entropy)) # performs entropy calculation in the initial state
c.add(gates.H(0)) # state is |+0> (entropy = 0)
c.add(gates.CallbackGate(entropy)) # performs entropy calculation after H
c.add(gates.CNOT(0, 1)) # state is |00> + |11> (entropy = 1))
c.add(gates.CallbackGate(entropy)) # performs entropy calculation after CNOT

# Execute the circuit using the callback
final_state = c()

# Draw the circuit:
print('Given this circuit:')
print(c.draw())

# tf to float:
formatted_output = [format(tensor.numpy(), '.3f') for tensor in entropy[:]]
print('\nEntropy calculation is performed after c() is executed:\n', formatted_output)
print('This is the entropy of the Bell state! :D\n')

# Prints the final state in Dirac form:
print('Final state:', final_state)

# Execute the circuit a second time
final_state = c()
formatted_output_2 = [format(tensor.numpy(), '.3f') for tensor in entropy[:]]
print('\nEntropy calculation when cicuit is executed for second time:\n', formatted_output_2)

Given this circuit:
q0: ─EE─H─EE─o─EE─
q1: ─EE───EE─X─EE─

Entropy calculation is performed after c() is executed:
 ['-0.000', '0.000', '1.000']
This is the entropy of the Bell state! :D

Final state: (0.70711+0j)|00> + (0.70711+0j)|11>

Entropy calculation when cicuit is executed for second time:
 ['-0.000', '0.000', '1.000', '-0.000', '0.000', '1.000']


The final state is the following Bell state:

$$|\Phi^+\rangle=\frac{|00\rangle+|11\rangle}{\sqrt{2}}$$

In [33]:
import numpy as np
1/np.sqrt(2)

0.7071067811865475

## Parametrized gates

Qibo gates such as rotations accept values for their free parameter. Once such gates are added in a circuit their parameters can be updated using the `qibo.models.circuit.Circuit.set_parameters()` method.

In [36]:
from qibo import Circuit, gates
# create a circuit with all parameters set to 0.
c = Circuit(3)

# All rotation angles are parametric and can be changed later
c.add(gates.RX(0, theta=0))
c.add(gates.RY(1, theta=0))
c.add(gates.CZ(1, 2))
c.add(gates.fSim(0, 2, theta=0, phi=0))
c.add(gates.H(2))

# Draw the circuit:
print('Given this circuit:')
print(c.draw())

# Prints the final state in Dirac form:
final_state = c()
print('\nFinal state:', final_state)

# set new values to the circuit's parameters
params = [0.123, 0.456, (0.789, 0.321)]
c.set_parameters(params)

# Prints the final state after changing parameters:
final_state = c()
print('\nFinal state:', final_state)


Given this circuit:
q0: ─RX───f───
q1: ─RY─o─|───
q2: ────Z─f─H─

Final state: (0.70711+0j)|000> + (0.70711+0j)|001>

Final state: (0.65746+0j)|000> + (0.71755+0j)|001> + (0.15255+0j)|010> + (0.1665+0j)|011> + -0.02983j|100> + -0.02983j|101> + -0.00692j|110> + -0.00692j|111>


Equivalently, set parameters `circuit.set_parameters()` with a dictionary or a flat list.

In [ ]:
c = Circuit(3)
g0 = gates.RX(0, theta=0)
g1 = gates.RY(1, theta=0)
g2 = gates.fSim(0, 2, theta=0, phi=0)
c.add([g0, g1, gates.CZ(1, 2), g2, gates.H(2)])

# set new values to the circuit's parameters using a dictionary
params = {g0: 0.123, g1: 0.456, g2: (0.789, 0.321)}
c.set_parameters(params)
# equivalently the parameter's can be update with a list as
params = [0.123, 0.456, (0.789, 0.321)]
c.set_parameters(params)
# or with a flat list as
params = [0.123, 0.456, 0.789, 0.321]
c.set_parameters(params)

The following gates support parameter setting:

* `RX`, `RY`, `RZ`, `U1`, `CU1`: Accept a single `theta` parameter.

* `qibo.gates.fSim`: Accepts a tuple of two parameters (`theta`, `phi`).

* `qibo.gates.GeneralizedfSim`: Accepts a tuple of two parameters `(unitary, phi)`. Here `unitary` should be a unitary matrix given as an array or `tf.Tensor` of shape `(2, 2)`.

* `qibo.gates.Unitary`: Accepts a single `unitary` parameter. This should be an array or `tf.Tensor` of shape `(2, 2)`.

**Note** that a `np.ndarray` or a `tf.Tensor` may also be used in the place of a flat list. Using `qibo.models.circuit.Circuit.set_parameters()` is more efficient than recreating a new circuit with new parameter values.

**Get parameters:** The inverse method `qibo.models.circuit.Circuit.get_parameters()` is also available and returns a list, dictionary or flat list.

Hide a parametrized gate from the action of `qibo.models.circuit.Circuit.get_parameters()` and `qibo.models.circuit.Circuit.set_parameters()`` by setting the `trainable=False` during gate creation.

In [42]:
c = Circuit(3)
c.add(gates.RX(0, theta=0.123))
c.add(gates.RY(1, theta=0.456, trainable=True))
c.add(gates.fSim(0, 2, theta=0.789, phi=0.567))

print(c.get_parameters())
# prints [(0.123,), (0.789, 0.567)] ignoring the parameters of the RY gate

[(0.123,), (0.456,), (0.789, 0.567)]


This is useful when the user wants to freeze the parameters of specific gates during optimization.